In [1]:
import tensorflow as tf

In [2]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    print('--------------')

    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))

    print('--------------')

    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))

    print('--------------')

    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')

    all = sess.run(hypothesis, feed_dict={
                   X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))

0 5.0953703
200 0.51853144
400 0.42757493
600 0.35334387
800 0.2807967
1000 0.23355299
1200 0.21193078
1400 0.19387528
1600 0.17856306
1800 0.1654166
2000 0.15401211
--------------
[[1.3179448e-02 9.8681086e-01 9.6185804e-06]] [1]
--------------
[[0.7655661  0.20707418 0.02735966]] [0]
--------------
[[9.6862234e-09 3.2878012e-04 9.9967122e-01]] [2]
--------------
[[1.3179448e-02 9.8681086e-01 9.6185804e-06]
 [7.6556611e-01 2.0707418e-01 2.7359661e-02]
 [9.6862234e-09 3.2878012e-04 9.9967122e-01]] [1 0 2]


Fancy 한 softmax classification 구현하기

여기서의 특징은

1. cost함수를 만들때 softmax_cross_entropy_with_logits 라는 텐서플로우에서 제공하는 함수를 통해 바로 구현가능하다 (logits = W*X + b)

2. Y 헷을 그냥 한자리 숫자가 아닌 one_hot으로 만들어주기 위해 텐서플로우의 one_hot 함수를 쓴다 이때는 차원이 하나 더 늘어나므로 reshape를 통해 모양을 적절히 변경해주는 것이 필요하다

3. 실제 prediction한 값들을 one hot 인코딩을 해주는 함수는 argmax 함수이다


In [3]:
import numpy as np
tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape", Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

(101, 16) (101, 1)
one_hot Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape Tensor("Reshape:0", shape=(?, 7), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Step:     0	Loss: 4.685	Acc: 16.83%
Step:   100	Loss: 0.466	Acc: 86.14%
Step:   200	Loss: 0.325	Acc: 91.09%
Step:   300	Loss: 0.250	Acc: 93.07%
Step:   400	Loss: 0.202	Acc: 97.03%
Step:   500	Loss: 0.169	Acc: 98.02%
Step:   600	Loss: 0.146	Acc: 99.01%
Step:   700	Loss: 0.128	Acc: 99.01%
Step:   800	Loss: 0.114	Acc: 100.00%
Step:   900	Loss: 0.103	Acc: 100.00%
Step:  1000	Loss: 0.094	Acc: 100.00%
Step:  1100	Loss: 0.087	Acc: 100.00%
Step:  1200	Loss: 0.080	Acc: 100.00%
Step:  1300	Loss: 0.075	Acc: 100.00%
Step:  1400	Loss: 0.070	Acc: 100.00%
Step:  1500	Loss: 0.066	Acc: 100.00%
Step:  1600	Loss: 0.062	Acc: 100.00%
Step:  1700	Loss: 0.059	Acc: 100.00%
Step:  1800	Lo